In [1]:
import requests
import pandas as pd
import sqlalchemy as db

In [64]:
class Imdb():
    
    def __init__(self):
        print('Criação para buscar lista dos filmes mais bem avaliados')
    
    def imdb(self):
        '''Criando a conexão através de API do site da Imdb, criando a tabela e editando'''
        
        response = requests.get('https://imdb-api.com/en/API/Top250Movies/k_4sz5q6a8')
        response.json()
        response.json().keys()
        results = response.json()
        df = pd.json_normalize(response.json(),'items')
        df.columns = ['ID', 'RANK', 'TITULO', 'TITULO COMPLETO', 'ANO DE LANÇAMENTO', 'IMAGEM', 'DIRETORs', 'NOTA DO PUBLICO', 'NUMERO DE VOTOS']
        df = df.drop(columns=['IMAGEM'])
        new = df["DIRETORs"].str.split("(", n = 1, expand = True)
        df["DIRETOR"]= new[0]
        new = df["DIRETORs"].str.split(",", n = 1, expand = True)
        df["ELENCO"]= new[1]
        df = df.drop(columns=['DIRETORs'])
        new = df["ID"].str.split('tt', n = 1, expand = True)
        df = df.drop(columns=['ID'])
        df['ID'] = new[1]
        df['ID'] = pd.to_numeric(df['ID'])
        df['ANO DE LANÇAMENTO'] = pd.to_numeric(df['ANO DE LANÇAMENTO'])
        df['NOTA DO PUBLICO'] = pd.to_numeric(df['NOTA DO PUBLICO'])
        df['RANK'] = pd.to_numeric(df['RANK'])
        df = df[['ID', 'RANK', 'TITULO', 'TITULO COMPLETO', 'ANO DE LANÇAMENTO', 'DIRETOR', 'ELENCO', 'NOTA DO PUBLICO', 'NUMERO DE VOTOS']]
        df['NUMERO DE VOTOS'] = pd.to_numeric(df['NUMERO DE VOTOS'])
        df['MEDIA DE VOTOS']=(df['NUMERO DE VOTOS']/df['NUMERO DE VOTOS'].sum())*100

        return df

In [65]:
df = Imdb()

Criação para buscar lista dos filmes mais bem avaliados


In [66]:
df = df.imdb()

In [67]:
df

,ID,RANK,TITULO,TITULO COMPLETO,ANO DE LANÇAMENTO,DIRETOR,ELENCO,NOTA DO PUBLICO,NUMERO DE VOTOS,MEDIA DE VOTOS
0,111161,1,The Shawshank Redemption,The Shawshank Redemption (1994),1994,Frank Darabont,"Tim Robbins, Morgan Freeman",9.2,2305958,1.723839
1,68646,2,The Godfather,The Godfather (1972),1972,Francis Ford Coppola,"Marlon Brando, Al Pacino",9.1,1591434,1.189691
2,71562,3,The Godfather: Part II,The Godfather: Part II (1974),1974,Francis Ford Coppola,"Al Pacino, Robert De Niro",9.0,1111857,0.831179
3,468569,4,The Dark Knight,The Dark Knight (2008),2008,Christopher Nolan,"Christian Bale, Heath Ledger",9.0,2269138,1.696314
4,50083,5,12 Angry Men,12 Angry Men (1957),1957,Sidney Lumet,"Henry Fonda, Lee J. Cobb",8.9,677818,0.506709
...,...,...,...,...,...,...,...,...,...,...
245,58946,246,The Battle of Algiers,The Battle of Algiers (1966),1966,Gillo Pontecorvo,"Brahim Hadjadj, Jean Martin",8.0,52037,0.038901
246,88247,247,The Terminator,The Terminator (1984),1984,James Cameron,"Arnold Schwarzenegger, Linda Hamilton",8.0,792074,0.592122
247,2991224,248,Tangerines,Tangerines (2013),2013,Zaza Urushadze,"Lembit Ulfsak, Elmo Nüganen",8.0,38672,0.028910
248,103639,249,Aladdin,Aladdin (1992),1992,Ron Clements,"Scott Weinger, Robin Williams",8.0,369495,0.276219


In [68]:
class bd():
    
    def __init__(self):
        print('Criando conexão com o banco de dados')
    
    def conectado(self):
        db_server = 'postgresql'
        user = 'postgres'
        password = 'admin'
        ip = 'localhost'
        db_name = 'melhores_filmes'
        engine = db.create_engine(f'{db_server}://{user}:{password}@{ip}/{db_name}')
        conn = engine.connect()
        return conn

In [69]:
conn = bd()

Criando conexão com o banco de dados


In [70]:
conn = conn.conectado()

In [71]:
df.to_sql(name='lista_de_filmes', con=conn, if_exists='replace', index=False )